In [1]:
import collections
import math
import string
import numpy as np
import pandas as pd
from itertools import permutations
from sympy.utilities.iterables import partitions
from sympy.functions.combinatorial.numbers import stirling
import matplotlib.pyplot as plt 
%matplotlib notebook
pd.set_option('precision', 7)

## Variables

In [2]:
pi = [1/2, 1/3, 1/6]
p1 = {3:1, 2:1, 1:1}
p2 = {3:2}
p3 = {6:1}
k=3
p_test = p3

**list_all_ints** is a helper function to type_probability

Input: the integer partition dictionary

Output: a list of the integers

```python
p = {3:1, 1:2}
col_names = [3, 1, 1]
```

In [3]:
def list_all_ints(p):
    col_names = []
    for i in p.keys(): # i = integer, m = multiplicity
        m = p[i]
        for x in range(m):
            col_names.append(i)
    return col_names

**type_3**

Input: 
- an integer partition p 
- pi, a list of probabilities per disease in k 
- k, the number of unique plaintext values/diseases

```python
p = {6:1}
pi = [.5, .333, .166]
k = 3
```

Output: 
- df, a dataframe. The first three columns are 'a', 'b', and 'c', the diseases.  The final column is the probability of the column's type. The rows represent the type of column.
- int_list, a expanded list of the integer partition. For example, if p = {3:2}, int_list = [3, 3]

An example of df:

|   -  | a | b | c | probs |
|---|--|---|---|-------|
| 0   | 6 | 0 | 0 | 0.0156250 |
| 1   | 0 | 0 | 6 | 0.0000214 |
| 2   | 0 | 6 | 0 | 0.0013717 |


Notes: 
- hardcoded for k = 3
- uses list_all_ints(p) as a helper function. 

In [4]:
def type_3(p, pi, k):
    int_list = list_all_ints(p)
    int_list.extend([0] * (k-len(int_list))) # pads list with 0s
    data = list(set(permutations(int_list, k)))
    df = pd.DataFrame(data = data, columns=['a', 'b','c'])
    df['prob'] = pow(pi[0], df['a']) * pow(pi[1], df['b']) * pow(pi[2], df['c'])
    return df, int_list

In [5]:
# Test of type_3 with different int partitions
my_df, my_int_list = type_3(p_test, pi, k)
print(my_df)

   a  b  c       prob
0  6  0  0  0.0156250
1  0  0  6  0.0000214
2  0  6  0  0.0013717


***choices_matrix()*** provides a matrix listing all the choices for a given integer partition and prior pi

Input:
- df: the dataframe with all possible types for the integer partition
- int_list: the expanded integer partition

Output:
- df_g, or dataframe_guess which contains the (not normalized) probabilities of every possible block in the integer partition being the respective diseases.

An example of df_g:

|   -  | a | b | c |
|---|--|---|---|
| 6   | 0.0156250 | 0.0013717 | 0.0000214 |
| 0   | 0.0013932 | 0.0156464 | 0.0169967 |
| 0   | 0.0013932 | 0.0156464 | 0.0169967 |

In [6]:
def choices_matrix(df, int_list):
    df_g = pd.DataFrame(0, index=int_list, columns=['a','b','c'])
    for i in int_list:
        df_g.loc[i,'a'] = df[df['a'] == i]['prob'].sum()
        df_g.loc[i,'b'] = df[df['b'] == i]['prob'].sum()
        df_g.loc[i,'c'] = df[df['c'] == i]['prob'].sum()
    return df_g

In [7]:
# Test of choices_matrix() w/type 3
my_df_guess = choices_matrix(my_df, my_int_list)
print(my_df_guess)

           a          b          c
6  0.0156250  0.0013717  0.0000214
0  0.0013932  0.0156464  0.0169967
0  0.0013932  0.0156464  0.0169967


***reduce_choices()*** reduces the size of the df_g matrix to only feasible answers and removes equal choices

Input:
- df_g, the matrix containing all of the possible choices
- p, the integer partition dictionary

Output:
- df_reduced_guess, a dataframe that indicates the possible unique choices an adversary choice make given an integer partition

In [8]:
def reduce_choices(df_g, p):
    data = []
    ### Minimize df
    # if p[k] (multiplicity) >1, only take one
    # only keep blocks > size 0 (where row index ! = 0)
    for k in p.keys(): # for every unique integer in p, copy a row in df_guess that matches the int
        if p[k] > 1: # choose first example if >1
            data.append(df_g.loc[k].iloc[0])# shouldn't need to .divide(p[k]))
        else:
            data.append(df_g.loc[k]) #copy the row, no edits
    df_reduced_guess = pd.DataFrame(data = data)
    return df_reduced_guess

In [9]:
my_reduced_df = reduce_choices(my_df_guess, p3)
my_reduced_df.head()

,a,b,c
6,0.015625,0.0013717,0.0000214


***choose_max()*** takes the reduced matrix and says which guess to make, along with the posterior vulnerability the integer partition provided

Input:
- df_reduced_guess, the reduced matrix of choices

Output:
- val, the contribution to the posterior vulnerability for one instance of one integer partition
- block_size, the block to guess about
- disease, the correct disease, or plaintext value, to guess

In [52]:
def choose_max(df_reduced_guess):
    ### Record max
    #val = df_reduced_guess.max().max() # contribution to the posterior vuln by this integer partition
    val = 5
    block_size = 0
    disease = 'a'
    return val, block_size, disease

In [53]:
# Tets of choose_max
my_val, my_block, my_disease = choose_max(my_reduced_df)
print("contribution of partition to posterior vuln: ", my_val)
print("The best choice is***")
print("block size: ", my_block)
print("disease: ", my_disease)

contribution of partition to posterior vuln:  5
The best choice is***
block size:  0
disease:  a


In [ ]:
df.max() # to max down dataframe column

In [ ]:
df.idxmax()

In [ ]:
df.max().max() # to max over entire dataframe

**calc_set_partitions()** calculates number of index partitions given an integer n and a integer partition p

In [ ]:
def calc_set_partitions(n,p):
    num = math.factorial(n)
    den = 1
    mul = 0
    for i in p.keys():
        m = p[i]
        den = den * pow(math.factorial(i), m) * math.factorial(m)
    frac = num / den
    return frac

### Tests

In [ ]:
my_int_list = [3, 3, 0]
my_list = list(set(permutations(my_int_list, k)))
print(my_list)

In [ ]:
for key in pi_dictionary:
    print("key: ", key)

In [ ]:
n = 6
k = 3
for p in partitions(n, m = k):
    print("p = ", p)
    print("p.keys()= ", p.keys())
    for k in p.keys():
        print("value: ", p[k])
    num = calc_set_partitions(n,p)
    print("index_partitions: ", num)

### Trash

**k_dictionary**

Input: pi, the probability distribution

Output: a dictionary with letters for keys. The letters represent individual diseases. Can be used to keep track of individual diseases throughout the gain function calculations.

Note-- Not currently in use

```python
pi = [1/2, 1/3, 1/6]
k_dict = {'a': 0.5, 'b': 0.33333, 'c':0.166666}
```

In [3]:
def k_dictionary(pi):
    return dict(zip(string.ascii_lowercase, pi))

In [ ]:
pi_dictionary = k_dictionary(pi)

print(pi_dictionary)

**type_1**

Input: 
- an integer partition p 
- pi, a list of probabilities per disease in k 

```python
p = {5:1, 1:1}
pi = [.5, .333, .166]
```

Output: a dataframe where the first three columns represent the type and the fourth is the type's probability

Notes: 
- hardcoded for k = 3
- uses col_names_p() as a helper function. 

In [ ]:
def type_1(p, pi):
    '''
    We use the int partition p to calculate 
    1. the number of bins and 
    2. the column names for the dataframe
    '''
    bins = sum(p.values()) #there will always be <=k 'bins' 
    col_names = list_all_ints(p)
    df = pd.DataFrame(list(permutations(pi, bins)), columns=col_names)
    df['prob'] = pow(df[col_names[0]],col_names[0]) *  pow(df[col_names[1]],col_names[1]) *  pow(df[col_names[2]],col_names[2])
    return df

In [39]:
df = type_1({3:1, 2:1, 1:1}, pi)
print(df)

           3          2          1       prob
0  0.5000000  0.3333333  0.1666667  0.0023148
1  0.5000000  0.1666667  0.3333333  0.0011574
2  0.3333333  0.5000000  0.1666667  0.0015432
3  0.3333333  0.1666667  0.5000000  0.0005144
4  0.1666667  0.5000000  0.3333333  0.0003858
5  0.1666667  0.3333333  0.5000000  0.0002572


In [ ]:
def type_2(p, pi, k):
    int_list = list_all_ints(p)
    int_list.extend([0] * (k-len(int_list))) # pads list with 0s
    df = pd.DataFrame(list(permutations(int_list, k)), columns=['a', 'b','c'])
    df['prob'] = pow(pi[0], df['a']) * pow(pi[1], df['b']) * pow(pi[2], df['c'])
    return df, int_list

In [40]:
# Test of type_2 with different int partitions
df2, int_list = type_2(p_test, pi, k)
df2.head()

,a,b,c,prob
0,6,0,0,0.0156250
1,6,0,0,0.0156250
2,0,6,0,0.0013717
3,0,0,6,0.0000214
4,0,6,0,0.0013717
